In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot as plt
import copy
import glob
import sklearn
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append('../../utils')

# importing
from emg_data_loading import *

In [84]:
# load nn predictions

AB_nn_sore = pd.read_csv('../prediction/amanda_nn_sore_daily_prediction.csv')
AB_nn_sore = AB_nn_sore[['date','pred']].rename(columns={'pred':'hooper_muscles_sore'})

AB_nn_tired = pd.read_csv('../prediction/amanda_nn_tired_daily_prediction.csv')
AB_nn_tired = AB_nn_tired[['pred']].rename(columns={'pred':'extra_muscles_tired'})

AB_nn_fatigue = pd.read_csv('../prediction/amanda_nn_fatigue_daily_prediction.csv')
AB_nn_fatigue = AB_nn_fatigue[['pred']].rename(columns={'pred':'cis_subjective_fatigue'})


AB_nn = pd.concat([AB_nn_sore, AB_nn_fatigue, AB_nn_fatigue],axis=1)
AB_nn['person']= 'Amanda'
AB_nn['Model'] = 'Neural network'


In [85]:
AB_nn

,date,hooper_muscles_sore,cis_subjective_fatigue,cis_subjective_fatigue,person,Model
0,2024-08-07,5.242895,3.661755,3.661755,Amanda,Neural network
1,2024-08-08,4.569926,3.661832,3.661832,Amanda,Neural network
2,2024-08-09,5.340290,3.661786,3.661786,Amanda,Neural network
3,2024-08-10,3.794274,3.661771,3.661771,Amanda,Neural network
4,2024-08-12,4.617346,3.661659,3.661659,Amanda,Neural network
5,2024-08-13,3.821817,3.661818,3.661818,Amanda,Neural network


In [86]:
# load rf predictions
AB_rf_sore = pd.read_csv('../prediction/amanda_rf_sore_daily_prediction.csv')
AB_rf_sore = AB_rf_sore[['date','pred']].rename(columns={'pred':'hooper_muscles_sore'})

AB_rf_tired = pd.read_csv('../prediction/amanda_rf_tired_daily_prediction.csv')
AB_rf_tired = AB_rf_tired[['pred']].rename(columns={'pred':'extra_muscles_tired'})


AB_rf_fatigue = pd.read_csv('../prediction/amanda_rf_fatigue_daily_prediction.csv')
AB_rf_fatigue = AB_rf_fatigue[['pred']].rename(columns={'pred':'cis_subjective_fatigue'})


AB_rf = pd.concat([AB_rf_sore, AB_rf_fatigue, AB_rf_fatigue],axis=1)
AB_rf['person']= 'Amanda'
AB_rf['Model'] = 'Random forest'

In [87]:
AB_rf

,date,hooper_muscles_sore,cis_subjective_fatigue,cis_subjective_fatigue,person,Model
0,2024-08-07,5.300724,4.875782,4.875782,Amanda,Random forest
1,2024-08-08,4.776178,4.851528,4.851528,Amanda,Random forest
2,2024-08-09,5.110354,4.909302,4.909302,Amanda,Random forest
3,2024-08-10,3.804934,4.170509,4.170509,Amanda,Random forest
4,2024-08-12,5.188367,4.722647,4.722647,Amanda,Random forest
5,2024-08-13,5.054689,5.180410,5.180410,Amanda,Random forest


In [88]:
# those are all predicitions for next day -> need to shift
predictions = pd.concat([AB_rf, AB_nn],axis=0)
predictions['date'] = (pd.to_datetime(predictions['date'])+timedelta(1))
predictions = predictions.reset_index().drop('index',axis=1)


In [91]:
survey_file = '../../../../data/survey_data/pre_processed_survey_features.csv'
survey = pd.read_csv(survey_file)
survey_sub = survey[['date','person','cis_subjective_fatigue','hooper_muscles_sore','extra_muscles_tired']]
survey_sub['date'] = pd.to_datetime(survey_sub['date'])
survey_sub =survey_sub[np.isin(survey_sub['date'], predictions['date'])]
survey_sub = survey_sub[survey_sub['person']=='Amanda']
survey_sub['Model'] = 'Ground truth'


/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_49022/2127544104.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_sub['date'] = pd.to_datetime(survey_sub['date'])


In [105]:
all_data = pd.DataFrame(np.concatenate([predictions,survey_sub[['date','hooper_muscles_sore','extra_muscles_tired','cis_subjective_fatigue','person','Model']]]))

In [102]:
# get mean
mean_df = df[pred_variables][:end_train].mean()
mean_df = mean_df.rename('mean')
mean_df = pd.merge(gb_ms_df[['index','measurement']],mean_df,how='left',right_index=True, left_on='measurement')
mean_df['model'] ="Always predict Mean"
mean_df = mean_df.rename(columns={'mean':'value'})

,date,hooper_muscles_sore,cis_subjective_fatigue,cis_subjective_fatigue,person,Model
0,2024-08-08,5.300724,4.875782,4.875782,Amanda,Random forest
1,2024-08-09,4.776178,4.851528,4.851528,Amanda,Random forest
2,2024-08-10,5.110354,4.909302,4.909302,Amanda,Random forest
3,2024-08-11,3.804934,4.170509,4.170509,Amanda,Random forest
4,2024-08-13,5.188367,4.722647,4.722647,Amanda,Random forest
5,2024-08-14,5.054689,5.180410,5.180410,Amanda,Random forest
6,2024-08-08,5.242895,3.661755,3.661755,Amanda,Neural network
7,2024-08-09,4.569926,3.661832,3.661832,Amanda,Neural network
8,2024-08-10,5.340290,3.661786,3.661786,Amanda,Neural network
9,2024-08-11,3.794274,3.661771,3.661771,Amanda,Neural network


In [19]:
.to_csv('../../../website/docs/amanda_signal_predictions.csv')

In [ ]:
# cis_subjective_fatigue	extra_muscles_tired	hooper_muscles_sore	model	index	person